Perform correlation analysis of the dataset to identify relationships that might indicate a causal connection with the bubble issue.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

import maps as m

# Load the Excel file
file_path = '2024_customer_feedback_bubble_test.xlsx'
selected_columns = ['Line', 'Blend', 'Height', 'Thickness',
                    'Color', 'Stamp', 'OutsideGroove', 'L1Only',
                    'Month', 'Day', 'Time']
df = pd.read_excel(file_path, usecols=selected_columns)

# THICKNESS: Convert all values in 'Thickness' column to a unique string value
df['Thickness'] = df['Thickness'].astype('str').map(m.thickness_map)
# COLOR: Convert all values in 'Color' column to a unique string value
df['Color'] = df['Color'].astype('str').map(m.color_map)
# STAMP: Convert all values in 'Stamp' column to a unique string value
df['Stamp'] = df['Stamp'].astype('str').map(m.stamp_map)
# OUTSIDE GROOME: Convert all values in 'Outside Groove' column to a unique string value
df['OutsideGroove'] = df['OutsideGroove'].astype('str').map(m.outside_groove_map)



# Perform a correlation analysis using the `corr()` method
correlation_matrix = df.corr()

# # Print the correlation matrix
# print(correlation_matrix)

# # Create a heatmap to visualize the correlations
# plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='nearest')
# plt.colorbar()
# plt.show()

# # Identify the most correlated variables
# most_correlated_variables = correlation_matrix.unstack().sort_values(ascending=False)[1:]

# # Print the most correlated variables
# print('Most correlated variables:')
# print(most_correlated_variables)

The correlation matrix will contain the Pearson correlation coefficients between each pair of variables. A value closer to 1 indicates a strong positive relationship, while values closer to -1 indicate a strong negative relationship. Values close to zero suggest no linear relationship between two variables.

In [ ]:
import pandas as pd

# Load the data into a DataFrame
data = pd.read_excel("2024_customer_feedback_bubble_test.xlsx")

# Calculate correlation coefficients between variables
corr_matrix = data.corr().abs()

# Print the correlation matrix
print(corr_matrix)